In [64]:
import cv2
import numpy as np

In [65]:
# bg image model
background = None

accumulated_weight = 0.5

# ROI coordinates
roi_top = 0
roi_bottom = 300
roi_right = 0
roi_left = 300

In [66]:
def calculate_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        # initialize the bg to the first frame
        background = frame.copy().astype("float")
        return None

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [67]:
def segment(frame, threshold=40):
    global background
    
    # difference between background and current frame
    difference = cv2.absdiff(background.astype("uint8"), frame)

    # Apply a threshold to the diff image so that we get the foreground
    _ , thresh = cv2.threshold(difference, threshold, 255, cv2.THRESH_BINARY)

    # Grab external contours
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None
    else:
        # Hand segment is the largest contour
        hand = max(contours, key=cv2.contourArea)

        return (thresh, hand)

In [68]:
def get_convex_hull(hand_segment):

    # Calculates the convex hull of the hand contour

    conv_hull = cv2.convexHull(hand_segment)
    return conv_hull

In [69]:
def get_extreme_points(conv_hull):

    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])

    return top, bottom, left, right

In [70]:
from sklearn.metrics import pairwise

def count_fingers(thresholded, hand_segment):


    conv_hull = get_convex_hull(hand_segment)

    top, bottom, left, right = get_extreme_points(conv_hull)

    # Center of the hand (palm center)
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2

    # Euclidean distances from center to extreme points
    distance = pairwise.euclidean_distances(
        [(cX, cY)],
        Y=[left, right, top, bottom]
    )[0]

    max_distance = distance.max()

    # Circular ROI parameters
    radius = int(0.8 * max_distance)
    circumference = 2 * np.pi * radius

    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)

    # Apply circular mask
    circular_roi = cv2.bitwise_and(
        thresholded,
        thresholded,
        mask=circular_roi
    )

    # Find contours in circular ROI
    contours, hierarchy = cv2.findContours(
        circular_roi.copy(),
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_NONE
    )

    # Finger counter
    count = 0

    for cnt in contours:

        (x, y, w, h) = cv2.boundingRect(cnt)

        # contour is above the wrist
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))

        # contour size is reasonable
        limit_points = ((circumference * 0.25) > cnt.shape[0])

        if out_of_wrist and limit_points:
            count += 1

    return count


In [71]:
cam = cv2.VideoCapture(0)

num_frames = 0

while True:

    ret, frame = cam.read()

    # flipping the frame
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    # gray scaling and blurring the ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # average of the first 60 frames
    if num_frames < 60:
        calculate_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "Getting background average.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
            
    else:
        
        hand = segment(gray)

        if hand is not None:
            
            thresholded, hand_segment = hand

            # Draw contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)

            # Count the fingers
            fingers = count_fingers(thresholded, hand_segment)

            # Display count
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            cv2.imshow("Thesholded", thresholded)

    # Draw ROI
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    num_frames += 1

    # Display the frame with segmented hand
    cv2.imshow("Finger Count", frame_copy)

    # Close the window with escape key
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()